# License Plate Detection

In [1]:
!pip install gdown ultralytics opencv-python wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.7/644.7 kB 15.6 MB/s eta 0:00:0000:01


In [2]:
import wandb
wandb.init(mode="disabled")

## Download the dataset

In [3]:
import os
import gdown
from zipfile import ZipFile
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    artifacts_root = "artifacts"
    source_URL = "https://drive.google.com/file/d/1lQLIIVJo9E7wZv6UAlVGJ-LEKppYqN2B/view?usp=sharing"
    local_data_file = "data.zip"
    unzip_dir = "artifacts"

class DataIngestion:
    def __init__(self):
        self.config = DataIngestionConfig()
        
    def download_file(self) -> None:
        """downloads file from the url"""
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.unzip_dir, exist_ok=True)
            print(
                f"Downloading data from {dataset_url} into the file {zip_download_dir}"
            )
            file_id = dataset_url.split('/')[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, str(zip_download_dir))
            print(f"Downloaded data from {dataset_url} into the file {zip_download_dir}")
            
        except Exception as e:
            raise e
        
    def extract_file(self) -> None:
        """ Extracts zip file into the data directory"""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, "r") as f:
            f.extractall(unzip_path)
            
data_ingestion = DataIngestion()
data_ingestion.download_file()
data_ingestion.extract_file()

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1lQLIIVJo9E7wZv6UAlVGJ-LEKppYqN2B
From (redirected): https://drive.google.com/uc?/export=download&id=1lQLIIVJo9E7wZv6UAlVGJ-LEKppYqN2B&confirm=t&uuid=88e5cec0-fb2e-4e43-89c2-a281b8697b49
To: /kaggle/working/data.zip
100%|██████████| 998M/998M [00:10<00:00, 94.1MB/s] 


Downloaded data from https://drive.google.com/file/d/1lQLIIVJo9E7wZv6UAlVGJ-LEKppYqN2B/view?usp=sharing into the file data.zip


In [4]:
import cv2 as cv
import os
import random
from ultralytics import YOLO

In [5]:
# Load in the pretrained model
os.makedirs('yolo-weights', exist_ok=True)
model_pretrained = YOLO('yolo-weights/yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 107MB/s]


In [6]:
# Testing the pretrained model
# !yolo task=detect mode=predict model=yolo-weights/yolov8n.pt source="https://ultralytics.com/images/bus.jpg"

In [7]:
# Training the model

# Reading the dataset metadata
dataset = "/kaggle/working/artifacts/data.yaml"

# Loading the model 
backbone = YOLO("yolo-weights/yolov8s.pt")

# Training the model
results = backbone.train(data=dataset, epochs=50, batch=32, )

100%|██████████| 21.5M/21.5M [00:00<00:00, 184MB/s]
Ultralytics YOLOv8.0.201 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
engine/trainer: task=detect, mode=train, model=yolo-weights/yolov8s.pt, data=/kaggle/working/artifacts/data.yaml, epochs=50, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, cla